<a href="https://colab.research.google.com/github/JuliaVazTostes/curso_data_science/blob/master/Modulo_2_Trabalhando_com_valores_ausentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Valores Ausentes

Se você está lidando com bases de dados do mundo real, pode ter certeza que estará lidando com dados incompletos ou valores ausentes.

Muitas vezes esses dados são inseridos por pessoas, manualmente. Há casos em que você já tinha séries históricas de anos, e em determinado momento alguém resolveu adicionar uma nova coluna.

Existem inúmeras técnicas para lidar com esse tipo de problema, e nesta aula vamos falar de duas das principais abordagens.

Antes, vamos importar os dados do Projeto +BIKE.


In [1]:
# importar os pacotes necessários
import pandas as pd

# importar o dataset
df = pd.read_csv("http://dl.dropboxusercontent.com/s/yyfeoxqw61o3iel/df_rides.csv")

# ver as primeiras entradas
df.head()

,user_gender,user_birthdate,user_residence,ride_date,time_start,time_end,station_start,station_end,ride_duration,ride_late
0,M,1971-06-08,NaN,2018-01-01,06:05:18,06:21:33,11 - Rodoviária 2,41 - Instituto de Artes,16.250000,0.0
1,M,1989-02-11,DF,2018-01-01,06:27:01,06:32:17,26 - Ministério da Saude,28 - CNMP - Conselho Nacional do Ministério Pú...,5.266667,0.0
2,M,1968-07-19,NaN,2018-01-01,06:29:33,06:44:57,11 - Rodoviária 2,43 - Biblioteca Central,15.400000,0.0
3,M,1991-12-19,NaN,2018-01-01,06:53:53,06:59:45,10 - Ministério dos Transportes,6 - Rodoviária,5.866667,0.0
4,M,1969-03-03,DF,2018-01-01,06:58:56,17:40:04,15 - Brasil 21,11 - Rodoviária 2,641.133333,1.0


## Identificando os valores ausentes

Já vimos anteriormente como identificar valores do tipo `NaN`. É indispensável identificar a quantidade de valores ausentes e qual a representatividade dessa quantidade frente ao total de entradas.

In [8]:
df.shape

(287322, 10)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287322 entries, 0 to 287321
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_gender     286926 non-null  object 
 1   user_birthdate  287321 non-null  object 
 2   user_residence  107417 non-null  object 
 3   ride_date       287322 non-null  object 
 4   time_start      287322 non-null  object 
 5   time_end        244037 non-null  object 
 6   station_start   287322 non-null  object 
 7   station_end     287322 non-null  object 
 8   ride_duration   214148 non-null  float64
 9   ride_late       214148 non-null  float64
dtypes: float64(2), object(8)
memory usage: 21.9+ MB


In [17]:
# df.describe() para retornar um resumo estatístico das variáveis numéricas

df.describe()

,ride_duration,ride_late
count,287322.000000,214148.000000
mean,25.913462,0.098829
std,51.271996,0.298433
min,3.000000,0.000000
25%,9.633333,0.000000
50%,14.200000,0.000000
75%,23.866667,0.000000
max,999.600000,1.000000


In [4]:
# ver a quantidade de valores ausentes - isnull retorna se a célula é verdadeiro ou falso para ausência

df.isnull()

,user_gender,user_birthdate,user_residence,ride_date,time_start,time_end,station_start,station_end,ride_duration,ride_late
0,False,False,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
287317,False,False,False,False,False,False,False,False,False,False
287318,False,False,True,False,False,False,False,False,False,False
287319,False,False,False,False,False,False,False,False,False,False
287320,False,False,True,False,False,False,False,False,False,False


In [6]:
# .sum() soma quantas células nulas existem em cada coluna

df.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [10]:
# ver a porcentagem de valores ausentes

df.isnull().sum() / df.shape[0]

user_gender       0.001378
user_birthdate    0.000003
user_residence    0.626144
ride_date         0.000000
time_start        0.000000
time_end          0.150650
station_start     0.000000
station_end       0.000000
ride_duration     0.254676
ride_late         0.254676
dtype: float64

## Excluir valores ausentes

Está é uma primeira opção quando você está lidando com valores nulos do tipo `NaN` no seu *DataFrame*.

Entretanto, ela tende a não ser ideal, pois por causa do valor de uma única célula, você elimina os dados existentes em outras colunas. Esta opção deve ser considerada no caso da quantidade de dados nulos serem pequenos a ponto de não ter representatividade no dataset

Eu consigo excluir tanto linhas com valores ausentes quanto colunas inteiras. Para que o Pandas saiba se a sua inteção é de eliminar linhas (`axis=0`) ou colunas (`axis=1`), você deve informar dentro do método `dropna()`.

In [11]:
# Eliminar linha
# eliminar todas as entradas onde existam valores ausentes em `user_gender`
# Para não atualizar em cima do dataframe, foi criado um novo dataframe: df_row_dropna

df_row_dropna = df.dropna(subset=['user_gender'], axis=0)

# comparar o antes e o depois
print("Antes:\t{}".format(df.shape))
print("Depois:\t{}".format(df_row_dropna.shape))

Antes:	(287322, 10)
Depois:	(286926, 10)


In [12]:
# Coluna
# Para não atualizar em cima do dataframe, foi criado um novo dataframe: df_cols_dropna

df_cols_dropna = df.dropna(axis=1,)

# comparar o antes e o depois
print("Antes:\t{}".format(df.shape))
print("Depois:\t{}".format(df_cols_dropna.shape))

Antes:	(287322, 10)
Depois:	(287322, 4)


## Preencher valores

Esta normalmente é a melhor opção, pois permite que você mantenha dados existentes em outras células.

Uma pergunta que normalmente surge é: "mas eu substituo o valor ausente por qual valor?". A resposta para essa pergunta é: depende.

Existem técnicas simples como usar valor mais frequente, media e mediana, assim como há técnicas mais avançadas que envolvem até mesmo o uso de modelos de *machine learning* cuja função é dizer qual valor usar nesses campos.

Vou mostrar como você pode usar a mediana para preencher os campos ausentes da coluna `ride_duration`, com o uso da função `fillna()`

In [13]:
# preencher valores ausentes em `ride_duration` com a mediana
# Foi criado uma variável chamada 'ride_duration_median'
# Na coluna ride_duration, será preenchidos os valores ausentes (.fillna) com a mediana (ride_duration_median) diretamente no dataframe que estamos chamando de df (inplace=True)

ride_duration_median = df.ride_duration.median()
df.ride_duration.fillna(ride_duration_median, inplace=True)

# ver valores ausentes

df.ride_duration.isnull().sum()

0

No caso da coluna `user_gender`, temos uma variável categórica. O ideal aqui é usar o valor mais frequente para o preenchimento dos valores. Valor ver qual aquele que tem maior recorrência e substituir diretamente na coluna.

In [14]:
# ver o valor mais frequente

df.user_gender.value_counts()

M    212608
F     74318
Name: user_gender, dtype: int64

In [15]:
# preencher os valores ausentes de user_gender com 'M'
df.user_gender.fillna('M', inplace=True)

# ver valores ausentes
df.user_gender.isnull().sum()

0